# Autograd

Hemos visto que para entrenar una red hay que calcular su función de pérdida (error entre la salida de la red y la etiqueta) y minimizar esta función de pérdida. Para minimizarla usamos los gradientes de la función de pérdida con respecto a los paŕametros de la red (ya que son estos los que podemos cambiar y harán que se minimice la función de pérdida).

Si la red consiste en $z = wx + b$ y el error se calcula como $loss = (y-z)^2$ los gradientes se calculan mediante:

$$\frac{\partial{loss}}{\partial{w}} = 2(y-z)\frac{\partial{z}}{\partial{w}} = 2(y-z)x$$
$$\frac{\partial{loss}}{\partial{b}} = 2(y-z)\frac{\partial{z}}{\partial{b}} = 2(y-z)$$

Pero si ahora la red cambia a $z = wx^2 + b$ el primer gradiente cambia a:

$$\frac{\partial{loss}}{\partial{w}} = 2(y-z)\frac{\partial{z}}{\partial{w}} = 2(y-z)x^2$$

Por tanto tiene que haber una manera automática de poder calcular los gradientes sin tener que calcularlos a mano para cada problema en particular

Pytorch resuelve esto mediante **Autograd**. Mediante `torch.autograd` Pytorch guarda las operaciones de las redes en un gráfico computacional

En el caso de la red

$$z = wx + b$$

Pytorch guarda el siguiente gráfico computacional

<div style="text-align:center;">
  <img src="https://pytorch.org/tutorials/_images/comp-graph.png" alt="comp-graph"> <!-- style="width:425px;height:626px;"> -->
</div>

En esta red, $w$ y $b$ son parámetros, que necesitamos optimizar. Por lo tanto, necesitamos poder calcular los gradientes de la función de pérdida con respecto a esas variables. Para hacer eso, establecemos la propiedad ``requires_grad`` de esos tensores.

 > **Nota**: Puede establecer el valor de ``requires_grad`` al crear un tensor, o más tarde mediante el método ``x.requires_grad_(True)``.

Veamos cómo lo hace Pytorch

Definimos una entrada y su etiqueta

In [1]:
import torch
seed = 1

torch.manual_seed(10*seed)

x = torch.randn(1)  # input tensor
y = torch.randn(1)  # expected output

print(f"x: {x}")
print(f"y: {y}")

x: tensor([-0.6014])
y: tensor([-1.0122])


Definimos los parámetros $w$ y $b$ aleatoriamente

In [2]:
torch.manual_seed(seed)
w = torch.randn(1, requires_grad=True)
b = torch.randn(1, requires_grad=True)

print(f"w: {w}")
print(f"b: {b}")

w: tensor([0.6614], requires_grad=True)
b: tensor([0.2669], requires_grad=True)


Por último definimos la salida y su función de pérdida

In [3]:
z = torch.matmul(x, w)+b
loss = torch.nn.functional.mse_loss(z, y, reduction='mean')

print(f"z: {z}")
print(f"Loss: {loss}")

z: tensor([-0.1308], grad_fn=<AddBackward0>)
Loss: 0.776868462562561


Calculemos a mano la salida y la función de pérdida para ver que está todo bien

$$z = wx + b$$

$$z = 0.6614·(-0.6014) + 0.2669 = -0.1309$$

Ahora la función de pérdida

$$loss = \frac{\sum_{i=1}^{N} \left(z-y\right)^2}{N}$$
$$loss = \frac{\left(-0.1309-(-1.0122)\right)^2}{1} = 0.7766$$

Ya tenemos la salida de la red y su fucnión de pérdida, para calcular los gradientes necesitamos calcular las derivadas de esta con respecto a $w$ y $b$

Para hacer esto en Pytorch simplemente tenemos que llamar al método `loss.backward()`, que calcula las derivadas de la función de pérdida hacia atrás

En el gráfico computacional cada nodo es una variable o parámetro y cada flecha es su salida, por lo que se calcula la derivada de cada salida con respecto las entradas o parámetros en función de las operaciones.

Una vez hemos hecho esto obtenemos las derivadas parciales (o gradientes) de la función de pérdida con respecto $w$ y $b$ llamando a `w.grad` y `b.grad`. Aquí Pytorch lo que hará será calcular estas derivadas parciales mediante la regla de la cadena, ya que tiene calculadas todas las derivadas en su gráfico computacional. Es decir realiza

$$\frac{\partial{loss}}{\partial{w}} = \frac{\partial{loss}}{\partial{z}}·\frac{\partial{z}}{\partial{w}}$$
$$\frac{\partial{loss}}{\partial{b}} = \frac{\partial{loss}}{\partial{z}}·\frac{\partial{z}}{\partial{b}}$$

Lo ejecutamos para ver que da

In [4]:
loss.backward()
print(f"gradiente de w: {w.grad}")
print(f"gradiente de b: {b.grad}")

gradiente de w: tensor([-1.0601])
gradiente de b: tensor([1.7628])


Vamos a calcularlo a mano para ver que obtenemos lo mismo

Gradiente con respecto a $w$

$$\frac{\partial{loss}}{\partial{w}} = \frac{\partial{loss}}{\partial{z}}·\frac{\partial{z}}{\partial{w}}$$

$$\frac{\partial{loss}}{\partial{w}} = \frac{2}{N}\left(\sum_{i=1}^{N} \left(z-y\right)\right)·x$$

$$\frac{\partial{loss}}{\partial{w}} = 2\left(-0.1309-(-1.0122)\right)·(-0.6014) = -1.4793$$


Gradiente con respecto a $b$


$$\frac{\partial{loss}}{\partial{b}} = \frac{\partial{loss}}{\partial{z}}·\frac{\partial{z}}{\partial{b}}$$
$$\frac{\partial{loss}}{\partial{b}} = \frac{2}{N}\left(\sum_{i=1}^{N} \left(z-y\right)\right)·1$$
$$\frac{\partial{loss}}{\partial{b}} = 2\left(-0.1309-(-1.0122)\right) = 1.7626$$

Se obtienen resultados similares.

 > Nota: No salen los mismos resultados, porque al hacer los cálculos a mano he redondeado, por lo que introduce errores

Como dijimos, si ahora la red cambia a $z = wx^2 + b$, habría que volver a hacer las derivadas y programar los cálculos de estas. Gracias a Autograd, llamando al método `loss.backgrd()` se realizan todas las derivadas gracias al gráfico computacional y ya solo nos falta obtener los gradientes mediante `w.grad` y `b.grad`